In [2]:
import numpy as np
import torch
import torchvision.transforms as T

In [3]:
def get_normalize(features: torch.Tensor):
    means = features.mean(axis = (0, 2, 3))
    stds = features.std(axis = (0, 2, 3))
    return (means, stds)

In [4]:
def get_augmentations(train: bool = True) -> T.Compose:
    #means = [125.3069180, 122.95039414, 113.86538318]
    #stds = [62.99321928, 62.08870764, 66.70489964]
    if train:
        return T.Compose(
            [
                #T.RandomRotation(degrees = 20),
                #T.RandomHorizontalFlip(),
                #T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                T.ToTensor(),
                #T.Normalize(mean = means, std = stds)
            ]
        )
    return T.Compose(
            [
                T.ToTensor(),
                #T.Normalize(mean = means, std = stds)
            ]
    )

In [5]:
from torch.utils.data import DataLoader
from torch import nn
@torch.inference_mode()
def predict(model: nn.Module, loader: DataLoader, device: torch.device):
    predictions = []
    model.eval()

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        _, y_pred = torch.max(output, dim = 1)
        predictions.append(y_pred)
    return torch.cat(predictions, dim = 0)

In [6]:
def create_simple_conv_cifar():
    return nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1), # 32 x 32 x 32
        nn.BatchNorm2d(32),
        nn.ReLU(),

        nn.MaxPool2d(2),
        nn.Dropout(p = 0.2),
        
        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1), # 16 x 16 x 64
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Flatten(),
        
        nn.Linear(16 * 16 * 64, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        
        nn.Linear(1024, 256),
        nn.BatchNorm1d(256),
        nn.Dropout(p = 0.2),
        nn.ReLU(),
        
        nn.Linear(256, 10)
    )

In [7]:
from tqdm import tqdm
def train_model(model: nn.Module, loader: DataLoader):
    model.train()
    train_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(loader, desc = 'train'):
        optimizer.zero_grad()

        output = model(x)

        loss = loss_fn(output, y)

        train_loss += loss.item()
        
        loss.backward()

        optimizer.step()

        _, y_pred = torch.max(output, dim = 1)
        correct += (y_pred == y).sum().item()
        total += y.size(0)

    train_loss /= len(loader)
    accuracy = correct / total
    return train_loss, accuracy

In [8]:
from IPython.display import clear_output

def plot_stats(
    train_loss: list[float],
    test_loss: list[float],
    train_accuracy: list[float],
    test_accuracy: list[float],
    title: str
):
    plt.figure(figsize = (16, 10))
    plt.title(title + 'Loss')
    plt.plot(train_loss, label = 'Train Loss')
    plt.plot(test_loss, label = 'Train Loss')
    plt.legend()
    plt.grid()
    plt.show()

    plt.figure(figsize = (16, 10))
    plt.title(title + 'Accuracy')
    plt.plot(train_accuracy, label = 'Train Accuracy')
    plt.plot(test_accuracy, label = 'Train Accuracy')
    plt.legend()
    plt.grid()
    plt.show()
    

In [69]:
from torch.optim import Adam
from torch.utils.data import DataLoader

loss_fn = nn.CrossEntropyLoss()
model = create_simple_conv_cifar()
optimizer = Adam(model.parameters(), lr = 1e-3)

transforms = T.Compose([
    T.ToTensor()
])

dataset_train = CIFAR10('../datasets', train = True, transform = transforms)
train_loader = DataLoader(dataset_train, batch_size = 128, shuffle = True, num_workers = 8)

In [9]:
dataset_test = CIFAR10('../datasets', train = False, transform = transforms)
test_loader = DataLoader(dataset_test, batch_size = 128, shuffle = False, num_workers = 8)

NameError: name 'CIFAR10' is not defined

In [79]:
dataset_test.data.shape

(10000, 32, 32, 3)

In [76]:
num_epochs = 10
for epoch in range(num_epochs):
    train_model(model, train_loader)

train:   0%|          | 0/391 [00:11<?, ?it/s]


KeyboardInterrupt: 

In [73]:
predicitons = predict(model, test_loader, torch.device('cpu'))

In [74]:
torch.save(predicitons, 'my_predictions.pt')

In [ ]:
def predict_tta(model: nn.Module, loader: DataLoader, device: torch.device, iterations: int = 2):
    for iteration in range(iterations):
        